### Main 페이지

### 1️⃣ Data_Upload

### 2️⃣ Table_Manager

### 3️⃣ Stock_Data_Anlysis

##### 제품별로 부진재고 시뮬레이션 간트차트 시각화 

In [ ]:
####################################
# 이전 간트차트 버전 (제품별로 간트차트 확인용)
####################################
# # no_sales 제외
# if "stop_reason" in gantt_df.columns:
#     gantt_df = gantt_df[gantt_df["stop_reason"] != "no_sales"].copy()

# # 날짜 컬럼 datetime으로 변환 (Plotly timeline용)
# for c in ["sell_start_date", "sell_end_date", "risk_entry_date"]:
#     if c in gantt_df.columns:
#         gantt_df[c] = pd.to_datetime(gantt_df[c], errors="coerce")

# # 판매 시작/끝 없는 행 제외
# gantt_df = gantt_df.dropna(subset=["sell_start_date", "sell_end_date"]).copy()

# st.write("### 🗓️ 제품별 배치 판매 간트 차트 (no_sales 제외)")

# # -----------------------------
# # 2) 제품 선택 UI
# # -----------------------------
# gantt_df["mat_label"] = gantt_df[MAT_COL].astype(str) + " | " + gantt_df[MAT_NAME_COL].astype(str)

# prod_list = sorted(gantt_df["mat_label"].unique())
# selected_prod = st.selectbox("제품 선택", options=["(전체)"] + prod_list)

# view_df = gantt_df if selected_prod == "(전체)" else gantt_df[gantt_df["mat_label"] == selected_prod].copy()

# # -----------------------------
# # 3) 간트 차트 (판매기간 + 부진재고 구간)
# # -----------------------------
# if view_df.empty:
#     st.info("표시할 데이터가 없습니다. (no_sales 제외 후 남은 배치가 없거나, sell_start/end가 비어있을 수 있어요.)")
# else:
#     # ✅ 만료일(expiry_date) 계산
#     view_df["expiry_date"] = pd.to_datetime(base_today) + pd.to_timedelta(view_df["init_days"], unit="D")

#     # ✅ 판매 구간
#     sales_bar = view_df.copy()
#     sales_bar["phase"] = "판매기간"
#     sales_bar = sales_bar.rename(columns={"sell_start_date": "x_start", "sell_end_date": "x_end"})

#     # ✅ 부진재고(잔존재고) 구간: remaining_qty > 0 인 배치만
#     sluggish_bar = view_df.copy()
#     sluggish_bar = sluggish_bar[sluggish_bar["remaining_qty"].fillna(0) > 0].copy()
#     sluggish_bar = sluggish_bar.dropna(subset=["risk_entry_date", "expiry_date"]).copy()
#     sluggish_bar["phase"] = "부진재고 구간"
#     sluggish_bar = sluggish_bar.rename(columns={"risk_entry_date": "x_start", "expiry_date": "x_end"})

#     # 합치기
#     plot_df = pd.concat([sales_bar, sluggish_bar], ignore_index=True)

#     # 배치 정렬 (유효기한 짧은 순 위로)
#     plot_df = plot_df.sort_values(["mat_label", "init_days"], ascending=[True, True])

#     # ✅ 색상 고정: 부진재고는 빨강
#     color_map = {
#         "판매기간": "#4C78A8",
#         "부진재고 구간": "#E45756"
#     }

#     fig = px.timeline(
#         plot_df,
#         x_start="x_start",
#         x_end="x_end",
#         y=BATCH_COL,
#         color="phase",
#         color_discrete_map=color_map,
#         hover_data={
#             MAT_COL: True,
#             MAT_NAME_COL: True,
#             "stop_reason": True if "stop_reason" in plot_df.columns else False,
#             "init_days": True if "init_days" in plot_df.columns else False,
#             "init_qty": True if "init_qty" in plot_df.columns else False,
#             "qty_sold": True if "qty_sold" in plot_df.columns else False,
#             "remaining_qty": True if "remaining_qty" in plot_df.columns else False,
#             "sold_days_total": True if "sold_days_total" in plot_df.columns else False,
#             "risk_entry_date": True if "risk_entry_date" in plot_df.columns else False,
#             "expiry_date": True if "expiry_date" in plot_df.columns else False,
#         },
#     )

#     fig.update_yaxes(autorange="reversed")
#     fig.update_layout(
#         height=550 if selected_prod == "(전체)" else 420,
#         margin=dict(t=30, b=10, l=10, r=10),
#         xaxis_title="기간",
#         yaxis_title="배치",
#         xaxis_title_font=dict(size=18, family="Arial Black"),
#         yaxis_title_font=dict(size=18, family="Arial Black"),
#         legend_title_text=""
#     )

#     fig.update_xaxes(
#     tickfont=dict(size=14, family="Arial Black")
#     )

#     fig.update_yaxes(
#         tickfont=dict(size=14, family="Arial Black")
#     )
#     st.plotly_chart(fig, use_container_width=True)

# # -----------------------------
# # 4) ✅ 간트 아래 요약 문장 출력 (제품 선택 시 배치별)
# # -----------------------------
# if selected_prod != "(전체)" and (not view_df.empty):
#     st.write("### 🧾 부진재고 요약 (배치별)")

#     summary_df = view_df[view_df["remaining_qty"].fillna(0) > 0].copy()
#     summary_df = summary_df.sort_values(["risk_entry_date", "init_days"], ascending=[True, True])

#     if summary_df.empty:
#         st.success("이 제품은 시뮬레이션 기준으로 D-180 시점에 부진재고로 남는 배치가 없습니다.")
#     else:
#         c1, c2, c3 = st.columns(3)
#         with c1:
#             st.metric("부진재고 배치 수", f"{len(summary_df)}개")
#         with c2:
#             st.metric("부진재고 수량 합계", f"{summary_df['remaining_qty'].sum():,.0f}개")
#         with c3:
#             first_date = summary_df["risk_entry_date"].min()
#             st.metric("가장 빠른 부진재고 진입일", first_date.strftime("%Y-%m-%d") if pd.notna(first_date) else "-")

#         st.write("#### 📌 배치별 문장 요약")
#         lines = []
#         for _, r in summary_df.iterrows():
#             b = r[BATCH_COL]
#             dt = r["risk_entry_date"]
#             qty = r["remaining_qty"]

#             dt_str = dt.strftime("%Y-%m-%d") if pd.notna(dt) else "-"
#             qty_str = f"{qty:,.0f}"

#             # (원하면 sold_days_total도 같이 보여줄 수 있음)
#             if "sold_days_total" in r and pd.notna(r["sold_days_total"]):
#                 sd = int(r["sold_days_total"])
#                 lines.append(
#                     f"- 배치 **{b}**는 **{dt_str}**부터 부진재고(D-180) 구간에 진입하며, "
#                     f"예상 잔량은 **{qty_str}개**입니다. (위험진입 전 판매일수: **{sd}일**)"
#                 )
#             else:
#                 lines.append(
#                     f"- 배치 **{b}**는 **{dt_str}**부터 부진재고(D-180) 구간에 진입하며, "
#                     f"예상 잔량은 **{qty_str}개**입니다."
#                 )

#         st.markdown("\n".join(lines))

#         with st.expander("📋 부진재고 배치 리스트 보기"):
#             show_cols = [
#                 BATCH_COL, "risk_entry_date", "expiry_date",
#                 "init_days", "init_qty", "qty_sold", "remaining_qty",
#                 "sold_days_total", "stop_reason"
#             ]
#             show_cols = [c for c in show_cols if c in summary_df.columns]
#             st.dataframe(summary_df[show_cols], use_container_width=True, height=260)